<a href="https://colab.research.google.com/github/mooncpark/nfl_fantasy_prediction/blob/main/Pipeline_Weekly_Fantasy_Football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nfl_data_py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

#preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#twitter scraper for sentiment analysis
import snscrape.modules.twitter as sntwitter
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


#modeling
from sklearn.svm import SVR, LinearSVC
from sklearn.linear_model import LinearRegression,LassoCV,Ridge,Lasso,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier


#eval
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, classification_report
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, RepeatedKFold, GridSearchCV
from sklearn.feature_selection import RFE, SelectKBest, chi2, f_regression, VarianceThreshold, mutual_info_regression, RFE
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from numpy import mean, std


#Viz
import seaborn as sns
import matplotlib.pyplot as plt

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# ALERT, if you don't want to be sitting around here forever (~15 mins per season) waiting for a twitter scraper, please make sure the three 2020-2022 tweets CSVs included in our repository are uploaded to your local session :)

# This pipeline has been broken into chunks - function definitions and then execution blocks where the functions are called. 

# Extraction Step 1 - Data Extraction from nfl_data_py

In [ ]:
#extract season data from nfl_data_py, both inputs taken as list (int of each season, list of columns)
#assumption is that current season will be updated after each week of games, if not sooner
#daily pipeline will repeatedly extract current season

def extract_all_season_data_dfs(seasons, pb_columns, wk_columns):
  print('Now extracting all dataframes from nfl_data_py')
  pbp_data = nfl.import_pbp_data(years=seasons, downcast=False, cache=False, alt_path=None)
  pbp_data = pbp_data[pb_columns] #limit to relevant columns, parameter for method call in nfl_data_py appears to not work
  pbp_data = pbp_data[(pbp_data['play'] == 1.0) & (pbp_data['season_type']=='REG')] #only non-special teams plays (pass or rush) and for regular season weeks (fantasy season)
  pbp_data.fillna(np.nan, inplace=True) #replace None types
  pbp_data['season'] = [int(g[:4]) for g in pbp_data['game_id']]

  week_data = nfl.import_weekly_data(years=seasons, columns=wk_columns, downcast=False) #limit to relevant columns
  week_data = week_data[(week_data['position'].isin(['QB','RB','WR','TE'])&(week_data['season_type']=='REG'))] #only fantasy relevant positions & for regular season weeks (fantasy season)


  sched_data = nfl.import_schedules(years=seasons) #contains game meta data

  rosters_data = nfl.import_rosters(years=seasons) #contains team level roster info and player profile data - might be a good way to split rookies, but unclear if updated in season

  injuries_data = nfl.import_injuries(years=seasons) #contains injury reports

  combine_data = nfl.import_combine_data(years=seasons, positions=['QB','RB','WR','TE']) #only fantasy relevant positions

  depth_data = nfl.import_depth_charts(years=seasons) #contains depth chart information

  latest_season = max(seasons) # to identify latest season in data

  return pbp_data, week_data, sched_data, rosters_data, injuries_data, combine_data, depth_data, latest_season

# Data Preparation Step 1 - Derive Additional Counting Stats from Play-by-Play Data

In [ ]:
# need to create some additional counting stats at the play level for additional potential features

def additional_pbp_counting_stats(pbp_df):
  print('Now using play-by-play data to identify specific fantasy relevant plays')
  pbp_df['compl_passes_20_yds'] = np.where(((pbp_df['complete_pass'] == 1.0) & (pbp_df['passing_yards'] >= 20.0)), 1, 0)
  pbp_df['compl_passes_40_yds'] = np.where(((pbp_df['complete_pass'] == 1.0) & (pbp_df['passing_yards'] >= 40.0)), 1, 0)
  pbp_df['compl_passes_60_yds'] = np.where(((pbp_df['complete_pass'] == 1.0) & (pbp_df['passing_yards'] >= 60.0)), 1, 0)
  pbp_df['pass_att_20_yds'] = np.where(((pbp_df['pass_attempt'] == 1.0) & (pbp_df['air_yards'] >= 20.0)), 1, 0)
  pbp_df['pass_att_40_yds'] = np.where(((pbp_df['pass_attempt'] == 1.0) & (pbp_df['air_yards'] >= 40.0)), 1, 0)
  pbp_df['pass_att_60_yds'] = np.where(((pbp_df['pass_attempt'] == 1.0) & (pbp_df['air_yards'] >= 60.0)), 1, 0)
  pbp_df['pass_att_goal'] = np.where(((pbp_df['pass_attempt']==1.0) & (pbp_df['yardline_100'] <= 5)), 1, 0)
  pbp_df['pass_att_red'] = np.where(((pbp_df['pass_attempt']==1.0) & (pbp_df['yardline_100'] <= 20)), 1, 0)
  pbp_df['rush_20_yds'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['rushing_yards'] >= 20.0)), 1, 0)
  pbp_df['rush_40_yds'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['rushing_yards'] >= 40.0)), 1, 0)
  pbp_df['rush_60_yds'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['rushing_yards'] >= 60.0)), 1, 0)
  pbp_df['goal_rush'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['yardline_100'] <= 5)), 1, 0)
  pbp_df['red_rush'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['yardline_100'] <= 20)), 1, 0)
  return pbp_df


Play By Play (PBP) Metrics not included in weekly data:

Player Level:  Goal line rushes, Red Zone Rushes, Red Zone Targets, Red Zone Pass Att, Deep Pass Att, Deep Compl Pass Attempts, Deep Targets, Big Plays

Team Offense Level: Total Valuable Plays, Total Rushes, Total Passes, Goal line rushes, Red Zone Rushes, Red Zone Targets, Red Zone Pass Att, Deep Pass Att, Deep Compl Pass Attempts, Deep Targets, Big Plays

Team Defense Level: Yards Allowed, Pass Attempts Allowed, Completions Allowed, Rushes allowed, YPA allowed, YPC allowed, YAC allowed, Long passes allowed, Long rushes allowed

# Data Preparation Step 2

In [ ]:
# need to collapse all pbp data to player id level, offense team level, and defense team level all at game level -- returns multiple dfs

def aggregate_pbp_data(pbp_df):
  print('Now aggregating all play-by-play data to week level and breaking into play type categories (pass, rush, receive, total offense, total defense)')
  player_game_df_pass = pbp_df.groupby(['game_id','season','week','passer_id','defteam']).agg({'compl_passes_20_yds':'sum','compl_passes_40_yds':'sum','compl_passes_60_yds':'sum',
                                                                                  'pass_att_20_yds':'sum','pass_att_40_yds':'sum','pass_att_60_yds':'sum',
                                                                                  'pass_att_goal':'sum','pass_att_red':'sum','sack':'sum'}).reset_index()
  player_game_df_pass = player_game_df_pass.rename(columns={'game_id':'game_id_p','defteam':'defteam_pas'}) #rename to avoid dupe column names

  player_game_df_rush = pbp_df.groupby(['game_id','season','week','rusher_id','defteam']).agg({'rush_20_yds':'sum','rush_40_yds':'sum','rush_60_yds':'sum',
                                                                                  'goal_rush':'sum','red_rush':'sum'}).reset_index()
  player_game_df_rush = player_game_df_rush.rename(columns={'game_id':'game_id_rus','defteam':'defteam_rus'}) #rename to avoid dupe column names


  player_game_df_receive = pbp_df.groupby(['game_id','season','week','receiver_id','defteam']).agg({'compl_passes_20_yds':'sum','compl_passes_40_yds':'sum','compl_passes_60_yds':'sum',
                                                                                    'pass_att_20_yds':'sum','pass_att_40_yds':'sum','pass_att_60_yds':'sum',
                                                                                    'pass_att_goal':'sum','pass_att_red':'sum'}).reset_index()
  player_game_df_receive.columns = ['game_id_rec','season','week','defteam_rec','receiver_id','catches_20_yards','catches_40_yards','catches_60_yards',
                                    'targets_20_yards','targets_40_yards','targets_60_yards','targets_goal','targets_red'] #rename receiving stats to avoid confusion with passing stats


  offense_game_df = pbp_df.groupby(['game_id','season','week','posteam','defteam']).agg({'compl_passes_20_yds':'sum','compl_passes_40_yds':'sum','compl_passes_60_yds':'sum',
                                                          'pass_att_20_yds':'sum','pass_att_40_yds':'sum','pass_att_60_yds':'sum',
                                                          'pass_att_goal':'sum','pass_att_red':'sum','goal_rush':'sum','red_rush':'sum',
                                                           'pass_attempt':'sum','rush_attempt':'sum'}).reset_index()
                      
  offense_game_df.columns = ['game_id_o','season','week','posteam','defteam_o','team_20_yard_compl_o','team_40_yard_compl_o','team_60_yard_compl_o',
                             'team_20_yard_pass_att_o','team_40_yard_pass_att_o','team_60_yard_pass_att_o','team_goal_pass_att_o',
                             'team_red_pass_att_o','team_goal_rush_att_o','team_red_rush_att_o','team_pass_att_o','team_rush_att_o'
                            ]             

  defense_game_df = pbp_df.groupby(['game_id','season','week','defteam']).agg({'compl_passes_20_yds':'sum','compl_passes_40_yds':'sum','compl_passes_60_yds':'sum',
                                                          'pass_att_20_yds':'sum','pass_att_40_yds':'sum','pass_att_60_yds':'sum',
                                                          'pass_att_goal':'sum','pass_att_red':'sum','goal_rush':'sum','red_rush':'sum',
                                                           'pass_attempt':'sum','complete_pass':'sum','rush_attempt':'sum','pass_touchdown':'sum',
                                                           'rush_touchdown':'sum','passing_yards':'sum','rushing_yards':'sum',
                                                           'sack':'sum','yards_after_catch':'sum'}).reset_index()
  

  defense_game_df['ypa_allowed'] = defense_game_df['passing_yards']/defense_game_df['pass_attempt'] # defense yards per attempt allowed
  defense_game_df['ypc_allowed'] = defense_game_df['rushing_yards']/defense_game_df['rush_attempt'] # defense yards per carry allowed
  defense_game_df['yac_per_compl_allowed'] = defense_game_df['yards_after_catch']/defense_game_df['complete_pass'] # defense yards after catch per catch allowed

  defense_game_df.columns = ['game_id','season','week','defteam','team_20_yard_compl_d','team_40_yard_compl_d','team_60_yard_compl_d',
                             'team_20_yard_pass_att_d','team_40_yard_pass_att_d','team_60_yard_pass_att_d','team_goal_pass_att_d',
                             'team_red_pass_att_d','team_goal_rush_att_d','team_red_rush_att_d','team_pass_att_d','team_compl_allowed',
                             'team_rush_att_d','pass_tds_allowed','rush_tds_allowed','passing_yards_allowed','rushing_yards_allowed','sacks_d','yac_allowed',
                             'ypa_allowed','ypc_allowed','yac_per_compl_allowed'
                            ]

  return player_game_df_pass, player_game_df_rush, player_game_df_receive, offense_game_df, defense_game_df                                                      

# Data Preparation Step 3

In [ ]:
# need to merge new player level stats & offense + defense team level stats, to weekly stats -- return master player-level weekly df

def create_master_weekly_player_data(week_data, player_game_df_pass, player_game_df_rush, player_game_df_receive, offense_game_df, defense_game_df):
  #add 0.5 PPR (point per reception) scoring

  week_data['fantasy_points_halfppr'] = week_data['fantasy_points_ppr'] - (0.5*week_data['receptions'])
  
  print('Now creating weekly historical positional rankings by each score format')

  weekly_pos_rankings = pd.DataFrame()
  for scoring_format in ['fantasy_points','fantasy_points_halfppr','fantasy_points_ppr']:
    scoring_df = week_data[['player_id','season','week','position',scoring_format]].copy()
    sort_scoring_df = scoring_df.sort_values(by=['season','week','position',scoring_format], ascending=[True,True,True,False])
    sort_scoring_df['wk_pos_rank-{}'.format(scoring_format)] = sort_scoring_df.groupby(['season','week','position']).cumcount()+1;sort_scoring_df
    sort_scoring_df = sort_scoring_df.set_index(keys=['player_id','season','week','position'])
    weekly_pos_rankings = pd.concat([weekly_pos_rankings,sort_scoring_df],join='outer',axis=1, ignore_index=False)
  weekly_pos_rankings = weekly_pos_rankings.reset_index()
  
  print('Now combining all player stats data at the player and week level')
  master_wk_df = pd.merge(week_data,player_game_df_pass,how='left', left_on=['player_id','season','week'], right_on=['passer_id','season','week'])
  master_wk_df = pd.merge(master_wk_df,player_game_df_rush,how='left', left_on=['player_id','season','week'], right_on=['rusher_id','season','week'])
  master_wk_df = pd.merge(master_wk_df,player_game_df_receive,how='left', left_on=['player_id','season','week'], right_on=['receiver_id','season','week'])
  master_wk_df = pd.merge(master_wk_df,offense_game_df,how='left', left_on=['season','week','recent_team'], right_on=['season','week','posteam'])

  master_wk_df.drop_duplicates(inplace=True)

  #add 0.5 PPR (point per reception) scoring

  master_wk_df['fantasy_points_halfppr'] = master_wk_df['fantasy_points_ppr'] - (0.5*master_wk_df['receptions'])

  #create rate data fields

  master_wk_df = master_wk_df.fillna(np.nan)
  master_wk_df = master_wk_df.replace(np.nan, 0)

  master_wk_df['ypa'] = master_wk_df['passing_yards']/master_wk_df['attempts'] #yards per pass attempt
  master_wk_df['air_ypa'] = master_wk_df['passing_air_yards']/master_wk_df['attempts'] #air yards per pass attempt
  master_wk_df['completion_per'] = master_wk_df['completions']/master_wk_df['attempts'] #passing completion percent
  master_wk_df['passing_td_rate'] = master_wk_df['passing_tds']/master_wk_df['attempts'] #passing td rate


  master_wk_df['goal_rush_team_per'] = master_wk_df['goal_rush']/master_wk_df['team_goal_rush_att_o'] #player percent of total team goaline rushes
  master_wk_df['red_rush_team_per'] = master_wk_df['red_rush']/master_wk_df['team_red_rush_att_o'] #player percent of total team redzone rushes
  master_wk_df['tot_rush_team_per'] = master_wk_df['carries']/master_wk_df['team_rush_att_o'] #player percent of total team rushes 

  master_wk_df['goal_targ_team_per'] = master_wk_df['targets_goal']/master_wk_df['team_goal_pass_att_o'] #player percent of total team goaline targets
  master_wk_df['red_targ_team_per'] = master_wk_df['targets_red']/master_wk_df['team_red_pass_att_o'] #player percent of total team redzone rushes

  #print(master_wk_df.columns)
  # clean up individual weekly player performance df

  new_cols = ['player_id','player_display_name','position','recent_team','season','week','defteam_o',
              'season_type','passing_yards','passing_tds','interceptions','sacks','sack_fumbles','sack_fumbles_lost','passing_air_yards',
              'compl_passes_20_yds','compl_passes_40_yds','ypa','air_ypa','completion_per','passing_td_rate','pass_att_goal','pass_att_red',
              'carries','rushing_yards','rushing_tds','rushing_fumbles','rushing_first_downs','rush_20_yds','rush_40_yds',
              'goal_rush_team_per','red_rush_team_per','tot_rush_team_per',
              'receptions','targets','receiving_yards','receiving_tds','receiving_fumbles',
              'receiving_air_yards','receiving_yards_after_catch','receiving_first_downs','catches_20_yards','catches_40_yards',
              'target_share','air_yards_share','goal_targ_team_per','red_targ_team_per',
              'fantasy_points','fantasy_points_halfppr','fantasy_points_ppr'
              ]

  trim_master_wk_df = master_wk_df[new_cols].copy()
  #trim_master_wk_df = trim_master_wk_df.rename(columns={'defteam_o':'defteam'}) #rename to avoid dupe column names

  return trim_master_wk_df, weekly_pos_rankings


#Data Preparation Step 4

In [ ]:
def create_schedule_df(sched_data, rosters_data, depth_data, injuries_data):
  #weekly game level meta data
  print('Now creating a dataframe containing schedule, team roster, depth chart and injuries at the player and week level')
  sched_cols = ['game_id', 'season', 'game_type', 'week', 'gameday', 'weekday',
              'gametime', 'div_game', 'roof', 'surface', 'temp', 'wind','away_team','home_team']

  trim_sched_data = sched_data[sched_cols].copy()
  trim_sched_data['teams'] = trim_sched_data[['away_team','home_team']].values.tolist()


  trim_sched_data = trim_sched_data.replace(np.nan,0)
  trim_sched_data = trim_sched_data[trim_sched_data['game_type'] == 'REG']
  trim_sched_data = trim_sched_data.explode(column='teams')
  trim_sched_data = trim_sched_data.drop(labels=['away_team','home_team'], axis=1)

  #depth chart
  depth_cols = ['gsis_id', 'season', 'week', 'game_type', 'position', 'club_code', 'depth_team']

  depth_data_trim = depth_data[depth_cols]
  depth_data_trim = depth_data_trim[depth_data_trim['game_type'] == 'REG']
  depth_data_trim = depth_data_trim[depth_data_trim['position'].isin(['QB','WR','RB','TE'])]
  depth_data_trim.sort_values(by=['season','week','club_code','gsis_id','depth_team'], ascending=True, inplace=True) #player duplicates in weekly depth chart

  #keep highest depth player entry for each weekly player record
  depth_data_trim.drop_duplicates(subset=['gsis_id', 'season', 'week', 'game_type', 'position', 'club_code'],keep='first',inplace=True) 
  #merge weekly game with depth chart

  sched_dep = pd.merge(trim_sched_data, depth_data_trim, how='left', left_on=['season', 'week', 'teams','game_type'], right_on=['season', 'week', 'club_code','game_type'])

  #player records via team roster at the end of each season, including player status
  roster_cols = ['player_id', 'player_name', 'season', 'status',
               'college', 'height', 'weight', 'years_exp', 'entry_year', 'rookie_year', 'draft_club',
               'draft_number']

  rosters_data_trim = rosters_data[roster_cols]
  rosters_data_trim = rosters_data_trim.drop_duplicates()

  #merge weekly game data with rosters

  sched_dep_ros = pd.merge(sched_dep, rosters_data_trim, how='left', left_on=['season', 'gsis_id'], right_on=['season', 'player_id'])

  #create simple season, player, week, team df

  team_df = sched_dep_ros[['player_id','season','week','teams']].copy()

  team_df = team_df.drop_duplicates()

  #add indicator for rookies

  sched_dep_ros['rookie'] = np.where(sched_dep_ros['rookie_year'] == sched_dep_ros['season'], 1,0)

  #merge with injuries data

  injuries_cols = ['season', 'week', 'gsis_id', 'report_status'] #report_status has None, Questionable, Doubtful, Out

  injuries_data.fillna(np.nan, inplace=True)

  injuries_data_fil = injuries_data[(injuries_data['game_type'] == 'REG') & (~injuries_data['report_status'].isnull())].copy() #only regular season injury reports and non

  injuries_data_fil = injuries_data_fil[injuries_cols].copy()

  sched_dep_ros_inj = pd.merge(sched_dep_ros, injuries_data_fil, how='left', left_on=['season', 'gsis_id','week'], right_on=['season', 'gsis_id','week'])

  return sched_dep_ros_inj, team_df


# Data Preparation Step 5

In [ ]:
#need to aggregate weekly data for weeks leading up to current week

def agg_player_stats_weeks_leading_up(sched_dep_ros_inj, trim_master_wk_df, pos_rankings_df, defense_game_df, current_season, current_week): #need to set current_season and current week
  print('Now aggregating player stats up to Current Season: {} and Current Week: {}'.format(str(current_season), str(current_week)))
  #add defteam for every game to sched df, to allow for all defense stats to be added even if player did not eventually play
  defteam_df = defense_game_df[['game_id','defteam']]
  defteam_df = defteam_df.drop_duplicates()
  sched_dep_ros_inj_def = pd.merge(sched_dep_ros_inj,defteam_df,how='left',on='game_id')
  sched_dep_ros_inj_def = sched_dep_ros_inj_def[sched_dep_ros_inj_def['defteam'] != sched_dep_ros_inj_def['teams']].copy()

  #merge sched df with weekly player master df

  full_player_df = pd.merge(sched_dep_ros_inj_def,weekly_mas_df,how='left',left_on=['player_id','season','week'],right_on=['player_id','season','week'])
  full_player_df['game_played'] = np.where(full_player_df['player_display_name'].isnull(), 0, 1) #derive game played flag based on pbp data accumulated

  #merge in weekly positional rankings
  pos_rankings_trim = pos_rankings_df[['player_id','season','week','wk_pos_rank-fantasy_points',
                                       'wk_pos_rank-fantasy_points_halfppr','wk_pos_rank-fantasy_points_ppr']].copy()

  full_player_df = pd.merge(full_player_df,pos_rankings_trim,how='inner',on=['player_id','season','week'])

  #filter to all player data before current season and current week
  full_player_df_lead = full_player_df[((full_player_df['season'] == current_season) & (full_player_df['week'] < current_week)) |
                                       (full_player_df['season'] < current_season)].copy()

  #only consider games where player accumulated stats
  full_player_active = full_player_df_lead[full_player_df_lead['game_played']==1].copy()

  stat_cols = ['passing_yards', 'passing_tds', 'interceptions', 'sacks',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'compl_passes_20_yds', 'compl_passes_40_yds', 'ypa', 'air_ypa',
       'completion_per', 'passing_td_rate', 'pass_att_goal', 'pass_att_red',
       'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
       'rushing_first_downs', 'rush_20_yds', 'rush_40_yds',
       'goal_rush_team_per', 'red_rush_team_per', 'tot_rush_team_per',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs',
       'catches_20_yards', 'catches_40_yards', 'target_share',
       'air_yards_share', 'goal_targ_team_per', 'red_targ_team_per',
       'wk_pos_rank-fantasy_points','wk_pos_rank-fantasy_points_halfppr',
       'wk_pos_rank-fantasy_points_ppr','fantasy_points','fantasy_points_halfppr',
       'fantasy_points_ppr'
       ]

  #group and average stats -- possible future enhancement: should we also include 1) stdev/variances? 2) avg or stdev/variance of fantasy points as features?)
  #also include total number of games played in each season leading up to current season/week
  full_player_games_played = full_player_active.groupby(['player_id','season'])['game_played'].sum().reset_index()
  full_player_active_grp = full_player_active.groupby(['player_id','season'])[stat_cols].mean().reset_index()

  full_player_active_grp = pd.merge(full_player_games_played,full_player_active_grp,how='inner',on=['player_id','season'])

  #calc standard deviation for the positional rank and score variables for each scoring format leading up to current week
  std_cols = ['wk_pos_rank-fantasy_points','wk_pos_rank-fantasy_points_halfppr',
       'wk_pos_rank-fantasy_points_ppr','fantasy_points','fantasy_points_halfppr',
       'fantasy_points_ppr']

  full_player_active_std = full_player_active.groupby(['player_id','season'])[std_cols].std().reset_index()

  full_player_active_std.columns = ['player_id','season'] + [s+'-std' for s in std_cols]

  #merge player leading up data with that player's data for the current week and through the rest of the season

  full_player_df_current = full_player_df[(full_player_df['season'] == current_season) & (full_player_df['week'] >= current_week)]

  full_player_active_grp.columns = ['player_id','season','game_played_sum'] + [s + '_avg' for s in stat_cols] #add 'avg' to each avg stat col, excl player id and season


  full_player_df_current_and_avg = pd.merge(full_player_df_current,full_player_active_grp,
                                          how='left',left_on=['player_id','season'],right_on=['player_id','season'])##need to remove season for weighting of previous seasons

  #create df with stdevs and avgs for weeks leading up and actuals for each player, season - only for pos rankings and points

  full_player_df_current_and_avg = pd.merge(full_player_df_current_and_avg,full_player_active_std,how='inner', on=['player_id','season'])

  #rename _avg cols used in prep for MLP classification (Modeling Prep Step 3) to parse scoring system correctly

  rename_col_dict = {'wk_pos_rank-fantasy_points_avg':'wk_pos_rank-fantasy_points-avg',
          'wk_pos_rank-fantasy_points_halfppr_avg':'wk_pos_rank-fantasy_points_halfppr-avg',
          'wk_pos_rank-fantasy_points_ppr_avg':'wk_pos_rank-fantasy_points_ppr-avg',
          'fantasy_points_avg':'fantasy_points-avg','fantasy_points_halfppr_avg':'fantasy_points_halfppr-avg',
          'fantasy_points_ppr_avg':'fantasy_points_ppr-avg'}

  full_player_df_current_and_avg = full_player_df_current_and_avg.rename(columns=rename_col_dict)

  #filter all opponent defense data to prior to current season and current week

  defense_game_df_lead = defense_game_df[((defense_game_df['season'] == current_season) & (defense_game_df['week'] < current_week)) |
                                       (defense_game_df['season'] < current_season)].copy()

  def_stat_cols = ['team_20_yard_compl_d',
       'team_40_yard_compl_d', 'team_60_yard_compl_d',
       'team_20_yard_pass_att_d', 'team_40_yard_pass_att_d',
       'team_60_yard_pass_att_d', 'team_goal_pass_att_d',
       'team_red_pass_att_d', 'team_goal_rush_att_d', 'team_red_rush_att_d',
       'team_pass_att_d', 'team_compl_allowed', 'team_rush_att_d',
       'pass_tds_allowed', 'rush_tds_allowed', 'passing_yards_allowed',
       'rushing_yards_allowed', 'sacks_d', 'yac_allowed', 'ypa_allowed',
       'ypc_allowed', 'yac_per_compl_allowed']

  #group and average defense stats (should we also include 1) stdev/variances?
  defense_game_df_grp = defense_game_df_lead.groupby(['defteam','season'])[def_stat_cols].mean().reset_index()

  #merge player data with avg opponent defense stats for the season up to that week

  full_player_df_current_avg_def = pd.merge(full_player_df_current_and_avg,defense_game_df_grp,
                                          how='left',left_on=['defteam','season'],right_on=['defteam','season'])##need to remove season for weighting of previous seasons

  #drop players with 0 as player_id

  full_player_df_current_avg_def = full_player_df_current_avg_def[full_player_df_current_avg_def['player_id'] != '0']

  #print(list(full_player_df_current_avg_def.columns))

  return full_player_df_current_avg_def
      



# Data Preparation Step 6

In [ ]:
def clean_and_split_vet_rook_feature_df(full_player_df_current_avg_def, current_season, current_week): #need to reset 
  #need to clean up columns and split into vets and rookies
  print('Now cleaning data into relevant data points for modeling avoiding leakage, split veterans and rookies')
  feature_cols = ['game_id','season','game_type','week','gameday','weekday','gametime','div_game',
          'roof','surface','temp','wind','teams','gsis_id','position_x','report_status','depth_team','player_id','player_name',
          'years_exp','rookie','game_played','game_played_sum','passing_yards_avg','passing_tds_avg','interceptions_avg','sacks_avg',
          'sack_fumbles_avg','sack_fumbles_lost_avg','passing_air_yards_avg','compl_passes_20_yds_avg',
          'compl_passes_40_yds_avg','ypa_avg','air_ypa_avg','completion_per_avg','passing_td_rate_avg','pass_att_goal_avg','pass_att_red_avg',
          'carries_avg','rushing_yards_avg','rushing_tds_avg','rushing_fumbles_avg','rushing_first_downs_avg','rush_20_yds_avg','rush_40_yds_avg',
          'goal_rush_team_per_avg','red_rush_team_per_avg','tot_rush_team_per_avg','receptions_avg','targets_avg','receiving_yards_avg','receiving_tds_avg',
          'receiving_fumbles_avg','receiving_air_yards_avg','receiving_yards_after_catch_avg','receiving_first_downs_avg','catches_20_yards_avg',
          'catches_40_yards_avg','target_share_avg','air_yards_share_avg','goal_targ_team_per_avg','red_targ_team_per_avg',
          'wk_pos_rank-fantasy_points-avg','wk_pos_rank-fantasy_points_halfppr-avg','wk_pos_rank-fantasy_points_ppr-avg',
          'fantasy_points-avg','fantasy_points_halfppr-avg','fantasy_points_ppr-avg','wk_pos_rank-fantasy_points-std','wk_pos_rank-fantasy_points_halfppr-std',
          'wk_pos_rank-fantasy_points_ppr-std','fantasy_points-std','fantasy_points_halfppr-std','fantasy_points_ppr-std','team_20_yard_compl_d',
          'team_40_yard_compl_d','team_20_yard_pass_att_d','team_40_yard_pass_att_d','team_goal_pass_att_d',
          'team_red_pass_att_d','team_goal_rush_att_d','team_red_rush_att_d','team_pass_att_d','team_compl_allowed','team_rush_att_d','pass_tds_allowed',
          'rush_tds_allowed','passing_yards_allowed','rushing_yards_allowed','sacks_d','yac_allowed','ypa_allowed','ypc_allowed','yac_per_compl_allowed',
           'wk_pos_rank-fantasy_points','wk_pos_rank-fantasy_points_halfppr','wk_pos_rank-fantasy_points_ppr',
          'fantasy_points','fantasy_points_halfppr','fantasy_points_ppr']

  clean_col_df = full_player_df_current_avg_def[feature_cols]

  #need to start encoding categorical variables

  #fillna with 0
  #clean_col_df.fillna(0,inplace=True)

  #to avoid future week data leakage, replace any future weeks:
  # 1) weather stats with nan 
  # 2) weekly depth chart positions with the current depth chart position
  # 3) injury report status with nan

  clean_data_leak_df = pd.DataFrame()
  for p in clean_col_df['player_id'][clean_col_df['season'] == current_season].tolist():
    player_df = clean_col_df[clean_col_df['player_id'] == p].copy()
    player_df_current_week = player_df[player_df['week'] == current_week].copy()
    player_df_past_current_week = player_df[player_df['week'] > current_week].copy()
    player_df_past_current_week['temp'] = np.nan
    player_df_past_current_week['wind'] = np.nan
    player_df_past_current_week['report_status'] = np.nan
    if len(player_df_current_week) > 0: #drop any player records that do not have record for current week
      player_df_past_current_week['depth_team'] = player_df_current_week['depth_team'].values[0]
      new_player_df = pd.concat([player_df_current_week,player_df_past_current_week], join='outer', axis=0)
      clean_data_leak_df = pd.concat([clean_data_leak_df,new_player_df], join='outer', axis=0)

  #split current season vets from current season rookies
  if len(clean_data_leak_df) > 0:
    clean_vet_df = clean_data_leak_df[(clean_data_leak_df['rookie'] == 0) | (clean_data_leak_df['season'] < current_season)].copy()
    clean_rookie_df = clean_data_leak_df[(clean_data_leak_df['rookie'] == 1) & (clean_data_leak_df['season'] == current_season)].copy()
  
  return clean_vet_df, clean_rookie_df
  

# Data Preparation Step 7

In [ ]:
def scrape_tweets_create_rookie_draft_profile(clean_rookie_df,combine_data,current_season):


  # Create a dictionary containing seasons and tuples containing the day after each draft concludes and the day before the season started
  # This time range is relevant for rookies being drafted in fantasy football prior to them actually taking the field (avoiding data leakage)

  season_dict = {2018:('2018-04-29','2018-09-05'),
               2019:('2019-04-28','2019-09-04'),
               2020:('2020-04-26','2020-09-09'),
               2021:('2021-05-02','2021-09-08'),
               2022:('2022-05-01','2022-09-07')
               }

  # Creating a list of well-known Fantasy Football Analyst Experts
  analysts = ['LateRoundQB','AndyHolloway','jasonffl','FFHitman','EvanSilva','scott_pianowski','daltondeldon','MikeClayNFL','numberFire','LizLoza_FF','andybehrens',
            'adamlevitan','adamrank','BrandonFunston','MattHarmon_BYB','TheFantasyPT','kyleynfl','NoisyHuevos','DaveKluge','Ihartitz','friscojosh','KevinColePFF',
            'ChrisRaybon','ScottBarrettDFB','BrandonHerFFB','The_Oddsmaker','MikeTagliereNFL','allinkid','dwainmcfarland','justinboone','thepme','Friscojosh','DBro_FFB',
            '_nickwhalen','jetpackgalileo']

  try: #recommended uploading csv with below name from GitHub repository to local directory, depending on what current season 2018-2022 you want.
  #single season takes ~15 mins to scrape
    print('Attempting to load {} Tweets from local session CSV'.format(str(current_season)))
    tweets_df = pd.read_csv('ff_analyst_tweets_current_season_{}.csv'.format(str(current_season)))

  except:
    print('Local session CSV not found, scraping Twitter for {} Tweets and saving CSV locally'.format(str(current_season)))
    attributes_container = []
    for a in tqdm(analysts):
      for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:{} since:{} until:{}'.format(a, season_dict[current_season][0],
                                                                                               season_dict[current_season][1])).get_items()):
        attributes_container.append([tweet.date, tweet.rawContent, a, str(current_season)])
    # Creating a dataframe from the tweets list above 
    tweets_df = pd.DataFrame(attributes_container, columns=["Date Created", "Tweets", "Author", "Season"])
    try:
      tweets_df.to_csv('ff_analyst_tweets_current_season_{}.csv'.format(str(current_season)), index=False)
      print('Tweets succesfully exported to csv')
    except:
      print('Error while exporting to csv')
  
  print('Now cleaning tweet data and looking for rookie references')
  #light cleaning on tweet data
  tweets_df['Tweets'] = tweets_df['Tweets'].str.lower().replace('&amp', '&')

  #define custom function to normalize primary names for joining on player across datasets

  def normalize_primary_nm(n):
    return str(n).lower().strip().replace("\\", '').replace("'", '').replace('.', '')

  #define custom function to derive different name variations likely in twitter references
  #done without regex for clearer visibility into different generated variations

  def name_aliases(n):
    l = []
    n = str(n).lower()
    n = n.strip()
    n = n.replace("\\", '')
  
    a = n.replace("'", '')
    b = n.replace("'", ' ')
    c = n.replace('-', '')
    d = n.replace('-', ' ')
    e = n.replace('.', '')
    f = n.replace('.',' ')

    for x in [n, a, b, c, d, e, f]:
      if x not in l:
       l.append(x)

    return l
  
  combine_data['name_norm'] = combine_data['player_name'].map(normalize_primary_nm)

  combine_data['aliases'] = combine_data['player_name'].map(name_aliases)

  fantasy_positions = ['QB','RB','WR','TE']

  # filter to current season, fantasy positions and those who were drafted

  current_season_combine_df = combine_data[(combine_data['season'] == current_season) & 
                                                       (combine_data['pos'].isin(fantasy_positions)) & (~combine_data['draft_year'].isnull())].copy()

  # iterate through list of players to find references
  player_names_list_of_lists = current_season_combine_df['aliases'].tolist()
  player_names = [i for s in player_names_list_of_lists for i in s]
  player_names = [n.replace("\\", '' ) for n in player_names]
  
  # filter tweets to those that have player references
  tweets_df['references'] = [[p for p in player_names if p in t] for t in tweets_df['Tweets']]
  tweets_df['ref_cnt'] = [len(r) for r in tweets_df['references']]

  tweets_df_ref = tweets_df[tweets_df['ref_cnt'] > 0].copy()
  print('Tweets from {} contain {} tweets with fantasy rookie references'.format(str(current_season),str(len(tweets_df_ref))))

  # let's explode the references data
  tweets_ref_expl = tweets_df_ref.explode(column='references', ignore_index=True)

  # let's explode the draft + conference data by all aliases
  current_season_combine_df_expl = current_season_combine_df.explode(column='aliases', ignore_index=True)

  #let's merge the draft + conference data with the tweet + ref data
  #let's only preserve players with references using inner join
  current_season_combine_ref = pd.merge(current_season_combine_df_expl, tweets_ref_expl, how='inner',
                                     left_on='aliases', right_on='references')
  
  print('Now assessing sentiment of rookie tweet references')
  # let's use VADER (Valence aware dictionary for sentiment reasoning) sentiment analysis model implemented in NLTK to assess sentiment in each tweet
  analyzer = SentimentIntensityAnalyzer()
  current_season_combine_ref['compound'] = [analyzer.polarity_scores(x)['compound'] for x in current_season_combine_ref['Tweets']]

  #let's aggregate our tweet data at the player level and clean up the final result
  field_list = ['player_name','draft_year', 'draft_round', 'draft_ovr', 'pos',
                'ht', 'wt', 'forty', 'bench', 'vertical', 'broad_jump', 'cone', 'shuttle',
                'name_norm','Tweets', 'Author','compound']
  field_list_for_agg = ['player_name','draft_year', 'draft_round', 'draft_ovr', 'pos',
                'ht', 'wt', 'forty', 'bench', 'vertical', 'broad_jump', 'cone', 'shuttle',
                'name_norm']

  current_season_combine_ref_fil = current_season_combine_ref[field_list].copy()

  #not every rookie completes all combine drills. to get rid of nans lets impute the mean for any NaNs

  current_season_combine_ref_fil_imp = current_season_combine_ref_fil.fillna(current_season_combine_ref_fil['forty'].mean())
  current_season_combine_ref_fil_imp = current_season_combine_ref_fil.fillna(current_season_combine_ref_fil['bench'].mean())
  current_season_combine_ref_fil_imp = current_season_combine_ref_fil.fillna(current_season_combine_ref_fil['vertical'].mean())
  current_season_combine_ref_fil_imp = current_season_combine_ref_fil.fillna(current_season_combine_ref_fil['broad_jump'].mean())
  current_season_combine_ref_fil_imp = current_season_combine_ref_fil.fillna(current_season_combine_ref_fil['cone'].mean())
  current_season_combine_ref_fil_imp = current_season_combine_ref_fil.fillna(current_season_combine_ref_fil['shuttle'].mean())

  current_season_combine_ref_fil_imp = current_season_combine_ref_fil_imp.groupby(field_list_for_agg)['compound'].agg(['count','mean','median']).reset_index()

  current_season_combine_ref_fil_imp['ht'] = current_season_combine_ref_fil_imp['ht'].fillna('0-0')
  current_season_combine_ref_fil_imp['ht_in'] = [str(float(str(h).split('-')[0])*12 + float(str(h).split('-')[1])) if '-' in str(h) else '0' for h in current_season_combine_ref_fil_imp['ht']]
  current_season_combine_ref_fil_imp['ht_in'] = current_season_combine_ref_fil_imp['ht_in'].replace('0', np.nan)

  final_draft_profile_cols = ['player_name','draft_year', 'draft_round', 'draft_ovr', 'pos',
                'ht_in', 'wt', 'forty', 'bench', 'vertical', 'broad_jump', 'cone', 'shuttle',
                'name_norm','count','mean','median']

  final_draft_profile = current_season_combine_ref_fil_imp[final_draft_profile_cols].copy()

  #let's give our expert sentiment fields more meaningful names

  final_draft_profile = final_draft_profile.rename(columns={'count':'tweet_ref_cnt','mean':'mean_compound_sentiment','media':'median_compound_sentiment'})

  #let's merge this draft profile with our clean rookie stat df

  clean_rookie_df['player_name_norm'] = clean_rookie_df['player_name'].map(normalize_primary_nm)



  final_draft_profile_clean_rookie_df = pd.merge(final_draft_profile,clean_rookie_df,how='inner',left_on=['name_norm'],
                                                 right_on=['player_name_norm'])
  
  final_draft_profile_clean_rookie_df = final_draft_profile_clean_rookie_df.drop_duplicates()
  
  return final_draft_profile_clean_rookie_df


In [ ]:
# combine_data[combine_data['ht'].isnull()]

# Execute Data Extraction and Preparation Portion of Pipeline

In [ ]:


#seasons = [2022]
#number of seasons to pull
seasons = [2020,2021,2022]

#relevant columns to extract from nfl_data_py
pbp_cols = ['play_id', 'game_id','home_team', 'away_team', 'season_type', 'week','posteam', 'posteam_type','defteam','side_of_field', 'yardline_100','goal_to_go',
            'yards_gained','play_type', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble','pass_length', 'pass_location', 'air_yards','sack','yards_after_catch', 'run_location', 'run_gap',
            'td_team', 'td_player_name','td_player_id','rush_attempt', 'pass_attempt','touchdown', 'pass_touchdown','rush_touchdown', 'return_touchdown',
            'fumble','complete_pass', 'passer_player_id', 'passer_player_name', 'passing_yards', 'receiver_player_id', 'receiver_player_name', 'receiving_yards', 
            'rusher_player_id', 'rusher_player_name', 'rushing_yards','fumbled_1_player_id', 'fumbled_1_player_name','success','pass','rush','play', 'passer_id',
            'rusher_id', 'receiver_id','fantasy_player_name', 'fantasy_player_id']

wk_cols = ['player_id', 'player_name', 'player_display_name', 'position','recent_team', 'season', 'week', 'season_type', 'completions', 'attempts',
           'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards','sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 
           'carries','rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs','receptions', 'targets', 'receiving_yards',
           'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch',
           'receiving_first_downs', 'receiving_2pt_conversions','target_share','air_yards_share','fantasy_points', 'fantasy_points_ppr']

#this will be updated to current week when this is productionized. step 5-7 only run for current season + current week here
current_season = 2022
current_week = 12

#Step 1: Extract various DFs from nfl_data_py library for seasons specified in seasons variable
pbp_data, week_data, sched_data, rosters_data, injuries_data, combine_data, depth_data, latest_season = extract_all_season_data_dfs(seasons, pbp_cols, wk_cols)

#Step 2: Extract and aggregate play by play data to create additional player counting stats and opposing defense stats
#Step 2.5: Clean and split into relevant game play category DFs (passing, rushing, receiving, total offense and total defense)
player_game_df_pass, player_game_df_rush, player_game_df_receive, offense_game_df, defense_game_df = aggregate_pbp_data(additional_pbp_counting_stats(pbp_data))

#Step 3: Create a weekly stat DF for each player based on their stats in each game play category as well as total offense and total defense and week pos rankings
weekly_mas_df, weekly_pos_rankings = create_master_weekly_player_data(week_data, player_game_df_pass, player_game_df_rush, player_game_df_receive, offense_game_df, defense_game_df)

#Step 4: Create season, week and player-level DF using schedules, rosters, depth charts and injury reports
sched_dep_ros_inj, team_df = create_schedule_df(sched_data, rosters_data, depth_data, injuries_data)

#Step 5: Aggregate player and opposing team defense average stats for past seasons and current season up to not including current week into one player df for the current week and weeks through end of season
full_player_df_current_avg_def = agg_player_stats_weeks_leading_up(sched_dep_ros_inj, weekly_mas_df, weekly_pos_rankings,
                                                                   defense_game_df, current_season=current_season, current_week=current_week)

#Step 6: Clean player df in preparation for supervised learning, limit data leakage, split into veterns versus rookies
clean_vet_df, clean_rookie_df = clean_and_split_vet_rook_feature_df(full_player_df_current_avg_def, current_season=current_season, current_week=current_week)

#Step 7: Create rookie draft profile for current season rookies to add as features in lieu of or combined with available current season data

final_draft_profile_clean_rookie_df = scrape_tweets_create_rookie_draft_profile(clean_rookie_df,combine_data,current_season=current_season)


Now extracting all dataframes from nfl_data_py
2020 done.
2021 done.
2022 done.
Now using play-by-play data to identify specific fantasy relevant plays
Now aggregating all play-by-play data to week level and breaking into play type categories (pass, rush, receive, total offense, total defense)
Now creating weekly historical positional rankings by each score format
Now combining all player stats data at the player and week level
Now creating a dataframe containing schedule, team roster, depth chart and injuries at the player and week level
Now aggregating player stats up to Current Season: 2022 and Current Week: 12
Now cleaning data into relevant data points for modeling avoiding leakage, split veterans and rookies
Attempting to load 2022 Tweets from local session CSV
Now cleaning tweet data and looking for rookie references
Tweets from 2022 contain 1112 tweets with fantasy rookie references
Now assessing sentiment of rookie tweet references


In [ ]:
#list(full_player_df_current_avg_def.columns)

In [ ]:
full_player_df_current_avg_def.head()

,game_id,season,game_type,week,gameday,weekday,gametime,div_game,roof,surface,...,team_rush_att_d,pass_tds_allowed,rush_tds_allowed,passing_yards_allowed,rushing_yards_allowed,sacks_d,yac_allowed,ypa_allowed,ypc_allowed,yac_per_compl_allowed
0,2022_12_BUF_DET,2022,REG,12,2022-11-24,Thursday,12:30,0,dome,fieldturf,...,28.500000,1.600000,1.700000,273.600000,155.100000,1.700000,106.600000,7.838479,5.288965,4.980590
1,2022_13_BUF_NE,2022,REG,13,2022-12-01,Thursday,20:15,1,outdoors,fieldturf,...,24.900000,1.200000,0.400000,211.600000,115.000000,3.600000,98.500000,5.821589,4.408976,5.317196
2,2022_14_NYJ_BUF,2022,REG,14,2022-12-11,Sunday,13:00,1,outdoors,a_turf,...,26.500000,0.900000,1.000000,221.100000,110.700000,3.200000,105.300000,6.113083,4.129911,4.857339
3,2022_15_MIA_BUF,2022,REG,15,2022-12-17,Saturday,20:15,1,outdoors,a_turf,...,24.500000,1.600000,1.200000,253.900000,118.400000,2.000000,123.800000,7.105898,4.758190,5.445364
4,2022_16_BUF_CHI,2022,REG,16,2022-12-24,Saturday,13:00,0,outdoors,grass,...,29.454545,1.090909,1.636364,205.454545,143.909091,1.363636,101.181818,7.166096,4.770511,5.638532


# Modeling Portion of Pipeline

In [ ]:
#let's store all columns relevant to each position for regression modeling purposes as a dict
#this has been updated as part of model tuning and EDA


reg_pos_col_dict = {
'QB':['season','week','player_id','player_name','report_status','depth_team','div_game','years_exp','game_played_sum',
      'passing_yards_avg','passing_tds_avg','interceptions_avg','sacks_avg',
      'sack_fumbles_avg','sack_fumbles_lost_avg','passing_air_yards_avg','compl_passes_20_yds_avg',
      'compl_passes_40_yds_avg','ypa_avg','air_ypa_avg','completion_per_avg','passing_td_rate_avg','pass_att_goal_avg','pass_att_red_avg',
      'carries_avg','rushing_yards_avg','rushing_tds_avg','rushing_fumbles_avg','rushing_first_downs_avg','rush_20_yds_avg','rush_40_yds_avg',
      'goal_rush_team_per_avg','red_rush_team_per_avg','tot_rush_team_per_avg',
      'team_20_yard_compl_d','team_40_yard_compl_d','team_20_yard_pass_att_d','team_40_yard_pass_att_d','team_goal_pass_att_d',
      'team_red_pass_att_d','team_goal_rush_att_d','team_red_rush_att_d','team_pass_att_d','team_compl_allowed','team_rush_att_d','pass_tds_allowed',
      'rush_tds_allowed','passing_yards_allowed','rushing_yards_allowed','sacks_d','yac_allowed','ypa_allowed','ypc_allowed','yac_per_compl_allowed',
      'fantasy_points','fantasy_points_halfppr','fantasy_points_ppr'],

'RB':['season','week','player_id','player_name','report_status','depth_team','div_game','years_exp','game_played_sum',
      'carries_avg','rushing_yards_avg','rushing_tds_avg','rushing_fumbles_avg','rushing_first_downs_avg','rush_20_yds_avg','rush_40_yds_avg',
      'goal_rush_team_per_avg','red_rush_team_per_avg','tot_rush_team_per_avg','receptions_avg','targets_avg','receiving_yards_avg','receiving_tds_avg',
      'receiving_fumbles_avg','receiving_air_yards_avg','receiving_yards_after_catch_avg','receiving_first_downs_avg','catches_20_yards_avg',
      'catches_40_yards_avg','target_share_avg','air_yards_share_avg','goal_targ_team_per_avg','red_targ_team_per_avg','team_goal_pass_att_d',
      'team_red_pass_att_d','team_goal_rush_att_d','team_red_rush_att_d','team_pass_att_d','team_rush_att_d','pass_tds_allowed',
      'rush_tds_allowed','passing_yards_allowed','rushing_yards_allowed','yac_allowed','ypc_allowed',
      'fantasy_points','fantasy_points_halfppr','fantasy_points_ppr'],

'WR':['season','week','player_id','player_name','report_status','depth_team','div_game','years_exp','game_played_sum',
      'carries_avg','rushing_yards_avg','rushing_tds_avg','rushing_fumbles_avg','rushing_first_downs_avg','rush_20_yds_avg','rush_40_yds_avg',
      'goal_rush_team_per_avg','red_rush_team_per_avg','tot_rush_team_per_avg','receptions_avg','targets_avg','receiving_yards_avg','receiving_tds_avg',
      'receiving_fumbles_avg','receiving_air_yards_avg','receiving_yards_after_catch_avg','receiving_first_downs_avg','catches_20_yards_avg',
      'catches_40_yards_avg','target_share_avg','air_yards_share_avg','goal_targ_team_per_avg','red_targ_team_per_avg','team_20_yard_compl_d',
      'team_40_yard_compl_d','team_20_yard_pass_att_d','team_40_yard_pass_att_d','team_goal_pass_att_d',
      'team_red_pass_att_d','team_goal_rush_att_d','team_red_rush_att_d','team_pass_att_d','team_compl_allowed','team_rush_att_d','pass_tds_allowed',
      'rush_tds_allowed','passing_yards_allowed','rushing_yards_allowed','sacks_d','yac_allowed','ypa_allowed','ypc_allowed','yac_per_compl_allowed',
      'fantasy_points','fantasy_points_halfppr','fantasy_points_ppr'],

'TE':['season','week','player_id','player_name','report_status','depth_team','div_game','years_exp','game_played_sum',
      'carries_avg','rushing_yards_avg','rushing_tds_avg','rushing_fumbles_avg','rushing_first_downs_avg','rush_20_yds_avg','rush_40_yds_avg',
      'goal_rush_team_per_avg','red_rush_team_per_avg','tot_rush_team_per_avg','receptions_avg','targets_avg','receiving_yards_avg','receiving_tds_avg',
      'receiving_fumbles_avg','receiving_air_yards_avg','receiving_yards_after_catch_avg','receiving_first_downs_avg','catches_20_yards_avg',
      'catches_40_yards_avg','target_share_avg','air_yards_share_avg','goal_targ_team_per_avg','red_targ_team_per_avg','team_20_yard_compl_d',
      'team_40_yard_compl_d','team_20_yard_pass_att_d','team_40_yard_pass_att_d','team_goal_pass_att_d',
      'team_red_pass_att_d','team_goal_rush_att_d','team_red_rush_att_d','team_pass_att_d','team_compl_allowed','team_rush_att_d','pass_tds_allowed',
      'rush_tds_allowed','passing_yards_allowed','rushing_yards_allowed','sacks_d','yac_allowed','ypa_allowed','ypc_allowed','yac_per_compl_allowed',
      'fantasy_points','fantasy_points_halfppr','fantasy_points_ppr'],

'ROOKIE':['draft_round','draft_ovr','ht_in',
       'wt','forty','bench','vertical','broad_jump','cone','shuttle',
       'name_norm','tweet_ref_cnt','mean_compound_sentiment'],

'POS_RANK':['wk_pos_rank-fantasy_points-avg',
      'wk_pos_rank-fantasy_points_halfppr-avg','wk_pos_rank-fantasy_points_ppr-avg',
      'fantasy_points-avg','fantasy_points_halfppr-avg','fantasy_points_ppr-avg'],

'STDEV':['wk_pos_rank-fantasy_points-std','wk_pos_rank-fantasy_points_halfppr-std',
      'wk_pos_rank-fantasy_points_ppr-std','fantasy_points-std','fantasy_points_halfppr-std',
      'fantasy_points_ppr-std']

}



#dict to set depth chart values for filtering
pos_depth_chart_dict = {'QB':1,'RB':2,'WR':2,'TE':1}

#set week cutoff to allow for aggregation of results in that season
week_cutoff = 3

# Modeling Setup Step 1 - Create DF containing all current week features for regression

In [ ]:
# define a function to return a DF containing all rows with current season and current week features
# this means we are trying to make predictions using all game data available up to that point
# week_cutoff is variable to only begin predictions in n week
# games_started_cutoff is variable to only consider players who have played in a min of n games so far that season

## need to run Extraction and Data Preparation Pipeline First
def create_all_current_week_df(sched_dep_ros_inj,weekly_mas_df,defense_game_df,combine_data,
                               seasons=seasons,week_cutoff=week_cutoff,games_played_cutoff=2):
  all_current_week_df_veteran = pd.DataFrame()
  all_current_week_df_rookie = pd.DataFrame()

  for s in seasons:
    try:
      # attempt to read in these DFs locally (takes ~2 seconds per week to compile DF)

      all_current_week_df_season_v = pd.read_csv('all_current_week_df_veteran_{}.csv'.format(str(s)))
      all_current_week_df_veteran = pd.concat([all_current_week_df_veteran,all_current_week_df_season_v],join='outer',axis=0)

      all_current_week_df_season_r = pd.read_csv('all_current_week_df_rookie_{}.csv'.format(str(s)))
      all_current_week_df_rookie = pd.concat([all_current_week_df_rookie,all_current_week_df_season_r],join='outer',axis=0)

    except:
      week_list = [x for x in list(sched_dep_ros_inj[sched_dep_ros_inj['season']==s]['week'].unique()) if x >= week_cutoff]
      print('generating feature dfs for the following weeks:',week_list)
      for w in tqdm(week_list):
        #execute aggregation up to current week function from data prep function
        full_player_df_current_avg_def = agg_player_stats_weeks_leading_up(sched_dep_ros_inj, weekly_mas_df, weekly_pos_rankings,
                                                                           defense_game_df, current_season=s, current_week=w)
        #only retain s season and w week row for each player (current season and current week at each loop)
        full_player_df_current_avg_def = full_player_df_current_avg_def[((full_player_df_current_avg_def['week']==w) &
                                                                         (full_player_df_current_avg_def['season']==s) &
                                                                         (full_player_df_current_avg_def['week']>=week_cutoff) &
                                                                         (full_player_df_current_avg_def['game_played_sum']>=games_played_cutoff)
                                                                         )].copy()
        clean_vet_df, clean_rookie_df = clean_and_split_vet_rook_feature_df(full_player_df_current_avg_def, current_season=s, current_week=w)
        final_draft_profile_clean_rookie_df = scrape_tweets_create_rookie_draft_profile(clean_rookie_df,combine_data,current_season=s)


        all_current_week_df_veteran = pd.concat([all_current_week_df_veteran,clean_vet_df],join='outer',axis=0)
        all_current_week_df_rookie = pd.concat([all_current_week_df_rookie,final_draft_profile_clean_rookie_df],join='outer',axis=0)
      
      all_current_week_df_veteran.to_csv('all_current_week_df_veteran_{}.csv'.format(str(s)))
      all_current_week_df_rookie.to_csv('all_current_week_df_rookie_{}.csv'.format(str(s)))

  return all_current_week_df_veteran, all_current_week_df_rookie


# Modeling Setup Step 2 - Create positional DFs by experience (vet and rookie)

In [ ]:
# define a function that takes vet and rookie dfs and returns clean positional vet and rookie dfs stored in a dictionary by vet/rookie and position
# also takes in two dicts, one to clean cols and one to set depth chart thresholds
# can be used for single current week or all current week dfs

def clean_and_split_by_pos_for_model(current_week_df_veteran, current_week_df_rookie, reg_pos_col_dict=reg_pos_col_dict,
                                     pos_depth_chart_dict=pos_depth_chart_dict):
  
  clean_pos_results_dict = {}

  ###new###
  clean_pos_results_dict['vet'] = {}
  clean_pos_results_dict['rookie'] = {}
  ###end new###
  fantasy_positions = ['QB','RB','WR','TE']
  current_week_df_rookie = current_week_df_rookie.rename(columns={'player_name_norm':'player_name'})
  for p in fantasy_positions:
    
    # filter by position
    fil_vet = current_week_df_veteran[current_week_df_veteran['position_x']==p].copy()
    fil_rook = current_week_df_rookie[current_week_df_rookie['position_x']==p].copy()

    # clean cols using passed dict
    fil_vet = fil_vet[reg_pos_col_dict[p]]
    fil_rook = fil_rook[reg_pos_col_dict[p]+reg_pos_col_dict['ROOKIE']]

    # filter by depth chart thresholds -- ensures only fantasy relevant players included
    fil_vet['depth_team'] = fil_vet['depth_team'].astype(int)
    fil_rook['depth_team'] = fil_rook['depth_team'].astype(int)
    fil_vet = fil_vet[fil_vet['depth_team'] <= pos_depth_chart_dict[p]]
    fil_rook = fil_rook[fil_rook['depth_team'] <= pos_depth_chart_dict[p]]

    # set indexes
    fil_vet = fil_vet.reset_index().set_index(keys=['season','week','player_id','player_name','report_status'])
    fil_rook = fil_rook.reset_index().set_index(keys=['season','week','player_id','player_name','report_status'])

    # fillna with 0

    fil_vet = fil_vet.fillna(0)
    fil_rook = fil_vet.fillna(0)

    # drop extra index
    fil_vet = fil_vet.drop(labels='index', axis=1)
    fil_rook = fil_rook.drop(labels='index', axis=1)

    ###updated###
    clean_pos_results_dict['vet'][p] = fil_vet
    clean_pos_results_dict['rookie'][p] = fil_rook
    ###end updated###

  return clean_pos_results_dict

# Modeling Setup Step 3 - Create feature df with boom-bust and buy-sell-hold labels for Multi Layer Perceptron classification training

In [ ]:



def create_boom_bust_buy_sell_dfs(current_week_df_veteran, current_week_df_rookie, reg_pos_col_dict=reg_pos_col_dict,
                                     pos_depth_chart_dict=pos_depth_chart_dict, boom_thres=7, bust_thres=-0.01, pos_rank_change_buy=-20,
                                 pos_rank_change_sell=20,seasons=seasons):
  
  scoring_formats = ['fantasy_points','fantasy_points_halfppr','fantasy_points_ppr']

  actual_cols = ['fantasy_points','fantasy_points_halfppr','fantasy_points_ppr',
                 'wk_pos_rank-fantasy_points','wk_pos_rank-fantasy_points_halfppr',
                'wk_pos_rank-fantasy_points_ppr']
  
  lead_avg_cols = ['wk_pos_rank-fantasy_points-avg',
      'wk_pos_rank-fantasy_points_halfppr-avg','wk_pos_rank-fantasy_points_ppr-avg',
      'fantasy_points-avg','fantasy_points_halfppr-avg','fantasy_points_ppr-avg']

  lead_stdev_cols = ['wk_pos_rank-fantasy_points-std','wk_pos_rank-fantasy_points_halfppr-std',
      'wk_pos_rank-fantasy_points_ppr-std','fantasy_points-std','fantasy_points_halfppr-std',
      'fantasy_points_ppr-std']

  # filter by depth chart thresholds -- ensures only fantasy relevant players included
  current_week_df_veteran['depth_team'] = current_week_df_veteran['depth_team'].astype(int)
  current_week_df_rookie['depth_team'] = current_week_df_rookie['depth_team'].astype(int)

  current_week_df_veteran['depth_team_check'] = current_week_df_veteran['position_x'].map(pos_depth_chart_dict)
  current_week_df_rookie['depth_team_check'] = current_week_df_rookie['position_x'].map(pos_depth_chart_dict)

  vet_fil = current_week_df_veteran[current_week_df_veteran['depth_team'] <= current_week_df_veteran['depth_team_check']].copy()
  rook_fil = current_week_df_rookie[current_week_df_rookie['depth_team'] <= current_week_df_rookie['depth_team_check']].copy()

  #key error n_stdevs-fantasy_points

  # calculate z scores for each current week observation

  for a in actual_cols:
    for df in [vet_fil, rook_fil]:
      df['n_stdevs-{}'.format(a)] = (df[a]/df['{}-avg'.format(a)])/(
          df['{}-std'.format(a)])

  # label booms or busts depending on z scores being above or below set thresholds   
  for df in [vet_fil, rook_fil]:
    for s in scoring_formats:
      df['{}-boom'.format(s)] = np.where(df['n_stdevs-{}'.format(s)] >= boom_thres, 1, 0)
      df['{}-bust'.format(s)] = np.where(df['n_stdevs-{}'.format(s)] < bust_thres, 1, 0)

  # to label buy-sell we need to calculate avg current+future weeks pos ranking
  buy_sell_labels_df = pd.DataFrame()
  for df in [vet_fil, rook_fil]:
    for s in seasons:
      season_df = df[df['season'] == s].copy()
      min_week = season_df['week'].min()
      max_week = season_df['week'].max()
      for w in np.arange(min_week, max_week+1, 1):
        #print('Labeling the Buy Sell Data for {} Season, Week {}'.format(str(s),str(w)))
        current_df = season_df[season_df['week'] >= w].copy()
        current_df_trim = current_df[['player_id','season','week','wk_pos_rank-fantasy_points-avg',
              'wk_pos_rank-fantasy_points_halfppr-avg','wk_pos_rank-fantasy_points_ppr-avg',
              'wk_pos_rank-fantasy_points','wk_pos_rank-fantasy_points_halfppr',
                'wk_pos_rank-fantasy_points_ppr']].copy()

        grouped_df = current_df_trim.groupby(['player_id','season','week','wk_pos_rank-fantasy_points-avg',
              'wk_pos_rank-fantasy_points_halfppr-avg','wk_pos_rank-fantasy_points_ppr-avg'])[['wk_pos_rank-fantasy_points',
                                                                                              'wk_pos_rank-fantasy_points_halfppr',
                                                                                              'wk_pos_rank-fantasy_points_ppr']].mean().reset_index()
        grouped_df = grouped_df.drop_duplicates()

        new_col_names = {'wk_pos_rank-fantasy_points':'remain_pos_rank-fantasy_points',
                         'wk_pos_rank-fantasy_points_halfppr':'remain_pos_rank-fantasy_points_halfppr',
                         'wk_pos_rank-fantasy_points_ppr':'remain_pos_rank-fantasy_points_ppr'}

        grouped_df = grouped_df.rename(columns=new_col_names)

        # let's calculate the difference in average player positions to identify buy or sell (negative change is good in this case, lower ranks are better)

        grouped_df['sell'] = np.where((grouped_df['remain_pos_rank-fantasy_points_halfppr']-
                                      grouped_df['wk_pos_rank-fantasy_points_halfppr-avg']) >= pos_rank_change_sell,1,0)
        
        grouped_df['buy'] = np.where((grouped_df['remain_pos_rank-fantasy_points_halfppr']-
                                      grouped_df['wk_pos_rank-fantasy_points_halfppr-avg']) < pos_rank_change_buy,1,0) 

        buy_sell_labels_df = pd.concat([buy_sell_labels_df,grouped_df], join='outer',axis=0)
  
  bb_label_cols = ['player_id','season','week','fantasy_points_halfppr-boom','fantasy_points_halfppr-bust']
  bs_label_cols = ['player_id','season','week','buy','sell']

  vet_labels = vet_fil[bb_label_cols].copy()
  rook_labels = rook_fil[bb_label_cols].copy()
  
  vet_rook_labels = pd.concat([vet_labels,rook_labels],join='outer',axis=0)  

  vet_rook_labels = vet_rook_labels.drop_duplicates()
  #vet_rook_labels = vet_labels.set_index(keys=['player_id','season','week'])

  buy_sell_labels_df_trim = buy_sell_labels_df[bs_label_cols].copy()

  final_labels_df = pd.merge(vet_rook_labels,buy_sell_labels_df_trim,how='inner',left_on=['player_id','season','week'],
                             right_on=['player_id','season','week'])

  final_labels_df = final_labels_df.drop_duplicates()

  final_labels_df['fantasy_points_halfppr-bust'] = np.where(final_labels_df['fantasy_points_halfppr-bust'] == 1, -1, 0)
  final_labels_df['sell'] = np.where(final_labels_df['sell'] == 1, -1, 0)

  final_labels_df['boom_or_bust'] = np.where(final_labels_df['fantasy_points_halfppr-boom'] == 1, 1, final_labels_df['fantasy_points_halfppr-bust'])
  final_labels_df['buy_or_sell'] = np.where(final_labels_df['buy'] == 1, 1, final_labels_df['sell'])

  return final_labels_df



In [ ]:
#list(all_current_week_df_veteran.columns)

# Modeling Setup Step 4 - Create Clean Labeled Feature DFs for Multilayer Perceptron Classifiers for boom-bust and buy-sell

In [ ]:
def create_clean_boom_bust_buy_sell_df(current_week_df_veteran, current_week_df_rookie, final_labels_df, seasons=seasons):
  
  # first need to get remaining season average defense stats for buy_sell and offensive features
  index_cols = ['player_id','player_name','season','week']

  off_avg_cols = ['position_x','depth_team','years_exp','rookie','passing_yards_avg','passing_tds_avg','interceptions_avg','sacks_avg',
          'sack_fumbles_avg','sack_fumbles_lost_avg','passing_air_yards_avg','compl_passes_20_yds_avg',
          'compl_passes_40_yds_avg','ypa_avg','air_ypa_avg','completion_per_avg','passing_td_rate_avg','pass_att_goal_avg','pass_att_red_avg',
          'carries_avg','rushing_yards_avg','rushing_tds_avg','rushing_fumbles_avg','rushing_first_downs_avg','rush_20_yds_avg','rush_40_yds_avg',
          'goal_rush_team_per_avg','red_rush_team_per_avg','tot_rush_team_per_avg','receptions_avg','targets_avg','receiving_yards_avg','receiving_tds_avg',
          'receiving_fumbles_avg','receiving_air_yards_avg','receiving_yards_after_catch_avg','receiving_first_downs_avg','catches_20_yards_avg',
          'catches_40_yards_avg','target_share_avg','air_yards_share_avg','goal_targ_team_per_avg','red_targ_team_per_avg',
          'wk_pos_rank-fantasy_points_halfppr-avg','fantasy_points_halfppr-avg','wk_pos_rank-fantasy_points_halfppr-std',
          'fantasy_points_halfppr-std']

  def_avg_cols = ['team_20_yard_compl_d','team_40_yard_compl_d','team_20_yard_pass_att_d','team_40_yard_pass_att_d','team_goal_pass_att_d',
          'team_red_pass_att_d','team_goal_rush_att_d','team_red_rush_att_d','team_pass_att_d','team_compl_allowed','team_rush_att_d','pass_tds_allowed',
          'rush_tds_allowed','passing_yards_allowed','rushing_yards_allowed','sacks_d','yac_allowed','ypa_allowed','ypc_allowed','yac_per_compl_allowed']


  pos_dict = {'QB':0,'RB':1,'WR':2,'TE':3}

  current_week_df_rookie = current_week_df_rookie.rename(columns={'player_name_norm':'player_name'})

  vet_fil = current_week_df_veteran[index_cols + off_avg_cols + def_avg_cols].copy()

  #lets prepare all the dfs for boom bust and buy sell

  rook_fil = current_week_df_rookie[index_cols + off_avg_cols + def_avg_cols].copy()

  bb_labels_fil = final_labels_df[['player_id','season','week','boom_or_bust']].copy()

  bs_labels_fil = final_labels_df[['player_id','season','week','buy_or_sell']].copy()

  full_df = pd.concat([vet_fil,rook_fil],join='outer',axis=0)

  full_df = full_df.drop_duplicates()

  full_df['position_x'] = full_df['position_x'].map(pos_dict)

  #create boom bust df with features up until the current week on, joined with the labels

  bb_df = pd.merge(full_df,bb_labels_fil,how='inner',left_on=['player_id','season','week'],right_on=['player_id','season','week'])

  bb_df = bb_df.drop_duplicates()

  bb_df = bb_df.set_index(keys=index_cols)

  #lets create the buy-sell df with defense features aggregated from future weeks (ie average of the average future opponents stats)

  #the offense stats will be past looking averages, while defensive stats will be forward opponent looking

  bs_df = pd.DataFrame()
  for s in seasons:
    season_df = full_df[full_df['season'] == s].copy()
    min_week = season_df['week'].min()
    max_week = season_df['week'].max()
    for w in np.arange(min_week, max_week+1, 1):
      current_df = season_df[season_df['week'] >= w].copy()
      current_df_trim = current_df[index_cols+def_avg_cols].copy()
      grouped_df = current_df_trim.groupby(index_cols)[def_avg_cols].mean().reset_index()

      #indicate that the cols are no longer past looking averages for a single team. they are averages for all future season opponents
      grouped_df.columns = index_cols + ['rem_'+c for c in def_avg_cols]

      bs_df = pd.concat([bs_df,grouped_df], join='outer',axis=0)
    
  bs_df = pd.merge(bs_df,bs_labels_fil,how='inner',left_on=['player_id','season','week'],right_on=['player_id','season','week'])

  bs_df = bs_df.drop_duplicates()

  bs_df = bs_df.set_index(keys=index_cols)

  return bb_df, bs_df
  

In [ ]:
#final_bb_bs_labels.shape

In [ ]:
#final_bb_bs_labels.sample(10)

In [ ]:
#bs_df.head()

In [ ]:
#list(all_current_week_df_veteran.columns)

# Execute Modeling Setup Pipeline

In [ ]:
#Step 1: Create Veteran and Rookie DFs containing only current week features and outcome variables (fantasy scores for each score setting)

all_current_week_df_veteran, all_current_week_df_rookie = create_all_current_week_df(sched_dep_ros_inj,weekly_mas_df,defense_game_df,combine_data,
                                                                                     seasons)

#Step 2: Create dictionary with clean feature DFs by vet/rookie by position, filtered by depth chart positions, with cols relevant to the position

all_current_dfs_dict = clean_and_split_by_pos_for_model(all_current_week_df_veteran, all_current_week_df_rookie, reg_pos_col_dict=reg_pos_col_dict,
                                     pos_depth_chart_dict=pos_depth_chart_dict)

#Step 3: Return dataframes that label booms-busts and buy-sells
#booms and busts are labeled on the number of standard deviations between their avg point average during prior season weeks and actual performance that week
#buy-sells are labeled using the difference of their mean weekly fantasy position rankings and their avg position ranking for the rest of the season

final_bb_bs_labels = create_boom_bust_buy_sell_dfs(all_current_week_df_veteran, all_current_week_df_rookie, reg_pos_col_dict=reg_pos_col_dict,
                                     pos_depth_chart_dict=pos_depth_chart_dict, boom_thres=7, bust_thres=-0.01, pos_rank_change_buy=-20,
                                     pos_rank_change_sell=20,seasons=seasons)

#Step 4: Create Clean Labeled Feature DFs for Multilayer Perceptron Classifiers for boom-bust and buy-sell
bb_df, bs_df  = create_clean_boom_bust_buy_sell_df(all_current_week_df_veteran, all_current_week_df_rookie,final_bb_bs_labels)


generating feature dfs for the following weeks: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


  0%|          | 0/15 [00:00<?, ?it/s]

Now aggregating player stats up to Current Season: 2020 and Current Week: 3
Now cleaning data into relevant data points for modeling avoiding leakage, split veterans and rookies
Attempting to load 2020 Tweets from local session CSV
Now cleaning tweet data and looking for rookie references
Tweets from 2020 contain 1040 tweets with fantasy rookie references
Now assessing sentiment of rookie tweet references


  7%|▋         | 1/15 [00:03<00:52,  3.75s/it]

Now aggregating player stats up to Current Season: 2020 and Current Week: 4
Now cleaning data into relevant data points for modeling avoiding leakage, split veterans and rookies
Attempting to load 2020 Tweets from local session CSV
Now cleaning tweet data and looking for rookie references
Tweets from 2020 contain 1040 tweets with fantasy rookie references
Now assessing sentiment of rookie tweet references


...

 88%|████████▊ | 14/16 [00:55<00:08,  4.35s/it]

Now aggregating player stats up to Current Season: 2022 and Current Week: 17
Now cleaning data into relevant data points for modeling avoiding leakage, split veterans and rookies
Attempting to load 2022 Tweets from local session CSV
Now cleaning tweet data and looking for rookie references
Tweets from 2022 contain 1112 tweets with fantasy rookie references
Now assessing sentiment of rookie tweet references


 94%|█████████▍| 15/16 [00:58<00:03,  3.96s/it]

Now aggregating player stats up to Current Season: 2022 and Current Week: 18
Now cleaning data into relevant data points for modeling avoiding leakage, split veterans and rookies
Attempting to load 2022 Tweets from local session CSV
Now cleaning tweet data and looking for rookie references
Tweets from 2022 contain 1112 tweets with fantasy rookie references
Now assessing sentiment of rookie tweet references


100%|██████████| 16/16 [01:01<00:00,  3.83s/it]


In [ ]:
#buy_sell.sample(10)

In [ ]:
##used for tuning boom/bust z score thresholds

#print(vet_boom_bust.groupby(['fantasy_points_halfppr-boom','week'])['player_id'].nunique())
#print(vet_boom_bust.groupby(['fantasy_points_halfppr-bust','week'])['player_id'].nunique())
#print(rook_boom_bust.groupby(['fantasy_points_halfppr-boom','week'])['player_id'].nunique())
#print(rook_boom_bust.groupby(['fantasy_points_halfppr-bust','week'])['player_id'].nunique())

In [ ]:
#used for tuning buy/sell thresholds

#print(buy_sell.groupby(['buy','week'])['player_id'].nunique())
#print(buy_sell.groupby(['sell','week'])['player_id'].nunique())

In [ ]:
# #example of how to access positional dfs
all_current_dfs_dict['vet']['RB'].head()

depth_team  \
season week player_id  player_name           report_status               
2020   3    00-0032780 Jordan Howard         NaN                     1   
            00-0033308 Matt Breida           NaN                     2   
            00-0030404 Chris Thompson        NaN                     2   
            00-0030578 Cordarrelle Patterson NaN                     1   
            00-0033556 Tarik Cohen           NaN                     1   

                                                            div_game  \
season week player_id  player_name           report_status             
2020   3    00-0032780 Jordan Howard         NaN                   0   
            00-0033308 Matt Breida           NaN                   0   
            00-0030404 Chris Thompson        NaN                   0   
            00-0030578 Cordarrelle Patterson NaN                   0   
            00-0033556 Tarik Cohen           NaN                   0   

                                                            years_exp  \
season week player_id  player_name           report_status              
2020   3    00-0032780 Jordan Howard         NaN                  4.0   
            00-0033308 Matt Breida           NaN                  3.0   
            00-0030404 Chris Thompson        NaN                  7.0   
            00-0030578 Cordarrelle Patterson NaN                  7.0   
            00-0033556 Tarik Cohen           NaN                  3.0   

                                                            game_played_sum  \
season week player_id  player_name           report_status                    
2020   3    00-0032780 Jordan Howard         NaN                        2.0   
            00-0033308 Matt Breida           NaN                        2.0   
            00-0030404 Chris Thompson        NaN                        2.0   
            00-0030578 Cordarrelle Patterson NaN                        2.0   
            00-0033556 Tarik Cohen           NaN                        2.0   

                                                            carries_avg  \
season week player_id  player_name           report_status                
2020   3    00-0032780 Jordan Howard         NaN                    6.5   
            00-0033308 Matt Breida           NaN                    6.0   
            00-0030404 Chris Thompson        NaN                    1.0   
            00-0030578 Cordarrelle Patterson NaN                    5.5   
            00-0033556 Tarik Cohen           NaN                    6.0   

                                                            rushing_yards_avg  \
season week player_id  player_name           report_status                      
2020   3    00-0032780 Jordan Howard         NaN                          5.5   
            00-0033308 Matt Breida           NaN                         29.5   
            00-0030404 Chris Thompson        NaN                          3.5   
            00-0030578 Cordarrelle Patterson NaN                         22.0   
            00-0033556 Tarik Cohen           NaN                         26.5   

                                                            rushing_tds_avg  \
season week player_id  player_name           report_status                    
2020   3    00-0032780 Jordan Howard         NaN                        1.0   
            00-0033308 Matt Breida           NaN                        0.0   
            00-0030404 Chris Thompson        NaN                        0.0   
            00-0030578 Cordarrelle Patterson NaN                        0.0   
            00-0033556 Tarik Cohen           NaN                        0.0   

                                                            rushing_fumbles_avg  \
season week player_id  player_name           report_status                        
2020   3    00-0032780 Jordan Howard         NaN                            0.0   
            00-0033308 Matt Breida           NaN                         

In [ ]:
#bb_df.head()

In [ ]:
#seasons[:-1]

# Modeling Step 1 - Function to train and return preds for MLP classifiers for boom-bust and buy_sell
Shoutout to Michael Fuchs of whom we used MLP code as listed at the following link https://michael-fuchs-python.netlify.app/2021/02/03/nn-multi-layer-perceptron-classifier-mlpclassifier/

Please note, this is modeling in somewhat of infancy that we added as an additional value add. Future tuning very much planned as the models very rarely identify scenarios. May require label tuning as well. Great learning experience here trying something novel :)

In [ ]:
def train_and_preds_classifier_bb(bb_df, seasons=seasons):

  index_cols = ['player_id','player_name','season','week']
  #let's train on 2020-2021 for both classifiers, test on 2022. will return 2022 preds for dashboarding
  # first for filtering we need to reset the indexes
  bb_df_mod = bb_df.reset_index()
  
  bb_df_mod = bb_df_mod.fillna(0)
  

  bb_train_df = bb_df_mod[bb_df_mod['season'].isin(seasons[:-1])].copy()
  bb_test_df = bb_df_mod[bb_df_mod['season'] == seasons[-1]].copy()


  #let's set indexes again

  bb_train_df = bb_train_df.set_index(keys=index_cols)
  bb_test_df = bb_test_df.set_index(keys=index_cols)


  #lets split into train and test for each of boom bust and buy sell

  bb_x_train = bb_train_df.loc[:, bb_train_df.columns != 'boom_or_bust']
  bb_y_train = bb_train_df['boom_or_bust']
  bb_x_test = bb_test_df.loc[:, bb_test_df.columns != 'boom_or_bust']
  bb_y_test = bb_test_df['boom_or_bust']


  #lets scale the data for bb

  sc=StandardScaler()

  scaler_bb = sc.fit(bb_x_train)
  bb_x_train_scaled = scaler_bb.transform(bb_x_train)
  bb_x_test_scaled = scaler_bb.transform(bb_x_test)

  #let's create the MLF model for bb
  mlp_clf_bb = MLPClassifier(hidden_layer_sizes=(5,2),
                        max_iter = 200,activation = 'relu',
                        solver = 'adam')
  
  mlp_clf_bb.fit(bb_x_train_scaled, bb_y_train)

  y_pred_bb = mlp_clf_bb.predict(bb_x_test_scaled)

  print('Accuracy for Boom Bust: {:.2f}'.format(accuracy_score(bb_y_test, y_pred_bb)))

  pred_df = bb_y_test.reset_index()

  pred_df = pred_df.drop(labels='boom_or_bust', axis=1)

  pred_df['boom_or_bust'] = y_pred_bb

  return pred_df

#just a copy of the above for bs
def train_and_preds_classifier_bs(bs_df, seasons=seasons):

  index_cols = ['player_id','player_name','season','week']

  bs_df_mod = bs_df.reset_index()
  bs_df_mod = bs_df_mod.fillna(0)
  bs_train_df = bs_df_mod[bs_df_mod['season'].isin(seasons[:-1])].copy()
  bs_test_df = bs_df_mod[bs_df_mod['season'] == seasons[-1]].copy()
  bs_train_df = bs_train_df.set_index(keys=index_cols)
  bs_test_df = bs_test_df.set_index(keys=index_cols)

  bs_x_train = bs_train_df.loc[:, bs_train_df.columns != 'buy_or_sell']
  #print(bs_x_train.columns)
  bs_y_train = bs_train_df['buy_or_sell']
  bs_x_test = bs_test_df.loc[:, bs_test_df.columns != 'buy_or_sell']
  bs_y_test = bs_test_df['buy_or_sell']

  #lets scale the data for bs

  sc=StandardScaler()

  scaler_bs = sc.fit(bs_x_train)
  bs_x_train_scaled = scaler_bs.transform(bs_x_train)
  bs_x_test_scaled = scaler_bs.transform(bs_x_test)

  #let's create the MLF model for bs
  mlp_clf_bs = MLPClassifier(hidden_layer_sizes=(5,2),
                        max_iter = 200,activation = 'relu',
                        solver = 'adam')
  
  mlp_clf_bs.fit(bs_x_train_scaled, bs_y_train)

  y_pred_bs = mlp_clf_bs.predict(bs_x_test_scaled)

  print('Initial Accuracy for Buy Sell: {:.2f}'.format(accuracy_score(bs_y_test, y_pred_bs)))

  #low accuracy so let's tune params

  #param_grid = {
  #  'hidden_layer_sizes': [(10,8,2), (12,10,5), (14,12,6)],
  #  'max_iter': [50, 100, 150],
  #  'activation': ['tanh', 'relu'],
  #  'solver': ['sgd', 'adam'],
  #  'alpha': [0.0001, 0.05],
  #  'learning_rate': ['constant','adaptive'],
  #}

  #grid = GridSearchCV(mlp_clf_bs, param_grid, n_jobs= -1, cv=5)
  #grid.fit(bs_x_train_scaled, bs_y_train)

  #print(grid.best_params_) 

  #grid_predictions = grid.predict(bs_x_test_scaled) 

  #print('Initial Accuracy for Buy Sell: {:.2f}'.format(accuracy_score(bs_y_test, grid_predictions)))

  pred_df = bs_y_test.reset_index()

  pred_df = pred_df.drop(labels='buy_or_sell', axis=1)

  pred_df['buy_or_sell'] = y_pred_bs

  return pred_df

# Modeling Step 2 - Function to Run Recursive Feature Elimination and Identify Optimal Regressions for each experience, position combination by scoring format

In [ ]:
## Set variables needed in the regression runs
clean_pos_results_dict = all_current_dfs_dict
regression_list = ['Linear','Ridge','Lasso','ElasticNet','DecisionTree']
models = {'Linear':LinearRegression(), 'Ridge': Ridge(), 'Lasso': Lasso(), 'ElasticNet': ElasticNet(), 'DecisionTree': DecisionTreeRegressor() }
scoring_formats = ['fantasy_points','fantasy_points_halfppr','fantasy_points_ppr']
positions = ['QB', 'RB', 'WR', 'TE']
experience = ['vet','rookie']
test_size = 0.2
random_state = 28

In [ ]:
### STEP1
##Run Recursive Feature Elimination and model for each combination of experience, position, and scoring format (24 combinations) 
##Run through each model type
##For each combination of experience, position, and scoring format. output the model name with the best (closest to zero) negative mean absolute error
def do_model(df, sformat, scoring_formats, model):
  y = df[sformat]
  drop_df = df.drop(labels=scoring_formats, axis=1) #drop all outcome variables
  X = drop_df
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
  rfe = RFE(estimator=model, n_features_to_select=10)
  pipeline = Pipeline(steps=[('s',rfe),('m',model)])
  # evaluate model
  cv = RepeatedKFold(n_splits=2, n_repeats=3, random_state=1)
  n_scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
  return mean(n_scores)

##dict with all identified optimal regressions for each combination of experience, position, and scoring format
regressions_mae = {}
for exp,d in clean_pos_results_dict.items():
  for sformat in scoring_formats:
    for position,df in d.items():
      
      # select proper df
      df = clean_pos_results_dict[exp][position]
      key = f'{exp}-{sformat}-{position}'
      results = {}
      for regression in regression_list:
        results[regression] = do_model(df,sformat,scoring_formats,models[regression])
      
      best_score = 0
      best_key = None
      for k,v in results.items():
        if best_key == None:
          best_score = v
          best_key = k
        else:
          if  v > best_score:
            best_key = k
            best_score = v


      regressions_mae[key] = (best_key, best_score)


In [ ]:
#Output from the code block above -- this is a dictionary that contains each experience, scoring, position format followed by 
# a tuple that shows the optimal model and the negative MAE associated with that model
print(regressions_mae)

{'vet-fantasy_points-QB': ('ElasticNet', -6.179444309255026), 'vet-fantasy_points-RB': ('ElasticNet', -4.902816606031439), 'vet-fantasy_points-WR': ('Ridge', -4.082270086538817), 'vet-fantasy_points-TE': ('Lasso', -3.5933153314941055), 'vet-fantasy_points_halfppr-QB': ('ElasticNet', -6.182736248111719), 'vet-fantasy_points_halfppr-RB': ('Lasso', -5.228690137944158), 'vet-fantasy_points_halfppr-WR': ('Ridge', -4.72177541811502), 'vet-fantasy_points_halfppr-TE': ('Lasso', -4.135575071399704), 'vet-fantasy_points_ppr-QB': ('ElasticNet', -6.186766162250624), 'vet-fantasy_points_ppr-RB': ('Lasso', -5.626067381942966), 'vet-fantasy_points_ppr-WR': ('Ridge', -5.412709675183467), 'vet-fantasy_points_ppr-TE': ('Ridge', -4.734878973812003), 'rookie-fantasy_points-QB': ('ElasticNet', -6.179444309255026), 'rookie-fantasy_points-RB': ('ElasticNet', -4.902816606031439), 'rookie-fantasy_points-WR': ('Ridge', -4.082270086538817), 'rookie-fantasy_points-TE': ('Lasso', -3.5933153314941055), 'rookie-fant

# Modeling Step 3 - Function to generate predictions for each optimal regression for each experience, position by scoring format

In [ ]:

#Variable split lets us know whether we want to train test split or train on the full df and predict using the current week df
#This will be relevant when we productionize into a weekly approach for future seasons

def do_fit_new(df, sformat, scoring_formats, model, split=True, current_df=None):
  # print(df.shape)
  if split == True:
    y = df[sformat]
    drop_df = df.drop(labels=scoring_formats, axis=1) #drop all outcome variables
    X = drop_df
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    rfe = RFE(estimator=model, n_features_to_select=10)
    pipeline = Pipeline(steps=[('s',rfe),('m',model)])
    # make predictions
    reg = model.fit(X_train, y_train)
    preds = reg.predict(X_test)
    return preds
  elif split == False:
    y_train = df[sformat]
    drop_df = df.drop(labels=scoring_formats, axis=1) #drop all outcome variables
    X_train = drop_df
    y_test = current_df[sformat]
    drop_df_current = current_df.drop(labels=scoring_formats, axis=1) #drop all outcome variables
    X_test = drop_df_current
    rfe = RFE(estimator=model, n_features_to_select=10)
    pipeline = Pipeline(steps=[('s',rfe),('m',model)])
    # make predictions
    reg = model.fit(X_train, y_train)
    preds = reg.predict(X_test)
    return preds

predictions = {}
for r,t in regressions_mae.items():
  parts = r.split('-')
  exp = parts[0]
  position = parts[2]
  df = clean_pos_results_dict[exp][position]
  selected_model = t[0]
  predictions[r] = do_fit_new(df, sformat, scoring_formats, models[selected_model])

# Modeling Step 4 - Use optimal regression function from Modeling Step 1 and predictions function from Step 3 to train iteratively up through current week in current season

In [ ]:
# this approach limits data leakage as each week is trained only using feature data up through that week
# pred_weeks is a list of weeks in the current season for which to generate score predictions

def curr_season_curr_week_preds(all_current_dfs_dict, pred_weeks, ypreds_bb, ypreds_bs, regressions_mae=regressions_mae,
                                current_season=current_season, week_cutoff=week_cutoff,
                                positions=positions, scoring_formats=scoring_formats, experience=experience, models=models, team_df=team_df):

  #ensure no values are provided before cutoff week in current season
  pred_weeks = [w for w in pred_weeks if w >= week_cutoff]
  first_pred_week = min(pred_weeks)
  last_pred_week = max(pred_weeks)

  #try to read in previous iteratively trained predictions for current_season
  try:
    print('Attempting to read in prior predictions for {} Season, Weeks {}-{}'.format(str(current_season),str(first_pred_week),str(last_pred_week)))
    current_season_preds = pd.read_csv('current_week_preds_{}_season_weeks_{}_{}.csv'.format(str(current_season),str(first_pred_week),str(last_pred_week)))
    return current_season_preds
  except:
    print('No matching CSV Found. Now executing training and predictions for each defined prediction week for each experience, position, scoring format combination')
    predictions_df = pd.DataFrame()
    for s in tqdm(scoring_formats):
      scoring_formats_df = pd.DataFrame()
      for w in pred_weeks:
        #week_pred_df = pd.DataFrame()
        for e in experience:
          for p in positions:
          
            #set key to lookup optimal regression in regressions_mae
            key = f'{e}-{s}-{p}'
            #need to reset_index to split df on week as df will contain all seasons rows at this point
            df = all_current_dfs_dict[e][p].reset_index()
            #split out training df with every record in prior seasons as well as through pred week (w)
            train_df = df[(df['season'] < current_season) | ((df['season'] == current_season) & (df['week'] < w))].copy()
            #split out test df with records in current season and week
            test_df = df[(df['season'] == current_season) & (df['week'] == w)].copy()

            #set indexes back
            train_df = train_df.set_index(keys=['season','week','player_id','player_name','report_status'])
            test_df = test_df.set_index(keys=['season','week','player_id','player_name','report_status'])
            
            #print('Week: {} Exp: {} Pos: {} Training DF has {} rows'.format(str(w),e,p,str(train_df.shape[0])))
            #print('Week: {} Exp: {} Pos: {} Test DF has {} rows'.format(str(w),e,p,str(test_df.shape[0])))

            opt_model = regressions_mae[key][0]
            preds = do_fit_new(train_df, s, scoring_formats, models[opt_model], split=False, current_df=test_df)

            test_df = test_df.reset_index()
            results_df = test_df[['season','week','player_id','player_name','report_status']].copy()

            #add preds as column titled with the scoring format
    
            results_df[s] = [round(p,2) for p in preds]
            results_df['position'] = p

            #need to override any players with an Out or Doubtful injury report status as 0, or negative predictions as 0
            results_df[s] = np.where(results_df[s] >= 0, results_df[s], 0)
            #results_df[s] = np.where(results_df['report_status'].isin(['Out','Doubtful']), 0, results_df[s])

            #veritcally concat all results to the scoring format level
            scoring_formats_df = pd.concat([scoring_formats_df,results_df],join='outer',axis=0)

      #set index
      scoring_formats_df = scoring_formats_df.set_index(keys=['season','week','player_id','player_name','report_status','position'])
      #horizontally concat all scoring_formats_df into the final preds_df to add each scoring column
      predictions_df = pd.concat([predictions_df,scoring_formats_df],join='outer',axis=1,ignore_index=False)
      #predictions_df = pd.concat([predictions_df,results_df],join='outer',axis=1,ignore_index=False)

    predictions_df = predictions_df.drop_duplicates()

    ###to comment out, just for dashboard testing###
    #predictions_df['boom_or_bust'] = random.choices(['boom','bust',np.nan], k=len(predictions_df),weights=[.1,.1,.8])
    #predictions_df['buy_or_sell'] = random.choices(['buy','sell',np.nan], k=len(predictions_df),weights=[.1,.1,.8])
    ###end comment out###

    predictions_df = predictions_df.reset_index()

    #merge with team dfs to get player teams

    predictions_df = pd.merge(predictions_df,team_df,how='inner',left_on=['player_id','season','week'], right_on=['player_id','season','week'])

    #let's merge with our boom or bust predictions
    predictions_df = pd.merge(predictions_df,ypreds_bb,how='inner',left_on=['player_id','player_name','season','week'], right_on=['player_id','player_name','season','week'])
    #print(predictions_df.columns)
    predictions_df = pd.merge(predictions_df,ypreds_bs,how='inner',left_on=['player_id','player_name','season','week'], right_on=['player_id','player_name','season','week'])
   # print(predictions_df.columns)
    #print(predictions_df.columns)

    predictions_df = predictions_df.drop_duplicates()

    #keep cols needed for dashboarding
    final_predictions_df = predictions_df[['season','week','player_name','teams','position','fantasy_points','fantasy_points_halfppr','fantasy_points_ppr',
                                           'report_status','boom_or_bust','buy_or_sell']].copy()
    #rename cols
    final_predictions_df.columns = ['season','week','player','team','position','standard','half_ppr','ppr',
                                           'injury_status','boom_or_bust','buy_or_sell']

    bb_norm_dict = {1:'boom',-1:'bust',0:np.nan}
    bs_norm_dict = {1:'buy',-1:'sell',0:np.nan}

    final_predictions_df['boom_or_bust'] = final_predictions_df['boom_or_bust'].map(bb_norm_dict)
    final_predictions_df['buy_or_sell'] = final_predictions_df['buy_or_sell'].map(bs_norm_dict)

    #final_predictions_df = final_predictions_df.fillna('')


    final_predictions_df = final_predictions_df.sort_values(by=['season','week','position','standard'], ascending=[True,True,True,False])
    #export predictions as a csv to avoid duplicating future runs
    print('Now exporting final prediction dataframe as CSV')
    final_predictions_df.to_csv('current_week_preds_{}_season_weeks_{}_{}.csv'.format(str(current_season),str(first_pred_week),str(last_pred_week)),header=True,index=False)

    return final_predictions_df


# Execute modeling pipeline

In [ ]:
#Generate preds for boom bust and buy sell
ypreds_bb = train_and_preds_classifier_bb(bb_df, seasons=seasons)
ypreds_bs = train_and_preds_classifier_bs(bs_df, seasons=seasons)

#Generate final consolidated preds file for dashboarding. hoorah!
preds_df = curr_season_curr_week_preds(clean_pos_results_dict,list(np.arange(week_cutoff,19,1)),ypreds_bb,ypreds_bs)

/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy for Boom Bust: 0.98
Initial Accuracy for Buy Sell: 0.63
Attempting to read in prior predictions for 2022 Season, Weeks 3-18
No matching CSV Found. Now executing training and predictions for each defined prediction week for each experience, position, scoring format combination


100%|██████████| 3/3 [00:19<00:00,  6.38s/it]

Now exporting final prediction dataframe as CSV


In [ ]:
preds_df.head()

,season,week,player,team,position,standard,half_ppr,ppr,injury_status,boom_or_bust,buy_or_sell
18,2022,3,Jalen Hurts,PHI,QB,22.80,22.83,22.86,,,
12,2022,3,Lamar Jackson,BAL,QB,21.16,21.18,21.19,,,
19,2022,3,Carson Wentz,WAS,QB,21.12,21.13,21.15,,,
8,2022,3,Josh Allen,BUF,QB,19.93,19.93,19.94,,,
16,2022,3,Derek Carr,LV,QB,19.54,19.55,19.56,,,


# Connection to Dashboard

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

auth.authenticate_user()

def write_to_sheet(df, sheet_id, sheet_range):

   # Create the service client
   service = build('sheets', 'v4')

   # Clear the existing values in the sheet
   service.spreadsheets().values().clear(
       spreadsheetId=sheet_id, range=sheet_range).execute()

   # Write the DataFrame to the sheet
   values = df.values.tolist()
   body = {
       'values': values
   }
   result = service.spreadsheets().values().update(
       spreadsheetId=sheet_id, range=sheet_range,
       valueInputOption='USER_ENTERED', body=body).execute()
   print('{0} cells updated.'.format(result.get('updatedCells')))

# The existing sheet we want to overwrite in Drive
sheet_id = '1KMhP6fn_4prOEYs284vBESojTH0J0hPkdwNQRb2EuC8'
sheet_range = 'project_data!A2:K'

# Execute the overwrite with data outputted from models in df (prediction_data)
write_to_sheet(preds_df[['season','week','player','team','position','standard',
                                'half_ppr','ppr','injury_status','boom_or_bust',
                                'buy_or_sell']], sheet_id, sheet_range)

32076 cells updated.


# End Pipeline, below for illustration and eval only

In [ ]:
# output predictions for future investigation/eval

predictions = {}
for r,t in regressions_mae.items():
  parts = r.split('-')
  exp = parts[0]
  position = parts[2]
  df = clean_pos_results_dict[exp][position]
  selected_model = t[0]
  predictions[r] = do_fit_new(df, sformat, scoring_formats, models[selected_model],split=True)

In [ ]:
#print out predictions of X_test -- this is a dictionary in the format of {experience-scoring-position: [predictions array]} for each of the 24 combinations
print(predictions['vet-fantasy_points-QB']) #this is just printing out predictions for veteran qbs in standard scoring, but the predictions dictionary has all of them

The two blocks above do all that is needed for getting the predictions on the test set

In [ ]:
#Step 1
#Create Veteran DFs
vet_qb_df = all_current_dfs_dict['vet']['QB']
vet_rb_df = all_current_dfs_dict['vet']['RB']
vet_wr_df = all_current_dfs_dict['vet']['WR']
vet_te_df = all_current_dfs_dict['vet']['TE']

#Create Rookie DFs
rook_qb_df = all_current_dfs_dict['rookie']['QB']
rook_rb_df = all_current_dfs_dict['rookie']['RB']
rook_wr_df = all_current_dfs_dict['rookie']['WR']
rook_te_df = all_current_dfs_dict['rookie']['TE']

The below boxes are just for data visualization -- I'm thinking for the blog I'll put in some of the graphs below -- I'm mainly just looking at one of the df combinations and showing some feature selection/correlation matrices etc for it

In [ ]:
#Step 2: Create training and testing data sets, split by different scoring options

#make X and y df for every vet, position, scoring combo -- should be 24 in total

#Veteran QBs
vet_qb_X = vet_qb_df.iloc[:,:-3]
vet_qb_y_stand = vet_qb_df['fantasy_points']


#Split into train and test sets for each of the 24 dataframes
X_train, X_test, y_train, y_test = train_test_split(vet_qb_X, vet_qb_y_stand, test_size=0.2, random_state=28)

In [ ]:
#Step 3: Quick Visualizations (2 Options) Showing Feature Importance -- Use graphs to justify n selection for selectkbest features in next step
#Additional Feature Selection Visualizations...Pick one of these two 

#Feature Selection For Correlation

# feature selection
f_selector = SelectKBest(score_func=f_regression, k='all')
# learn relationship from training data
f_selector.fit(X_train, y_train)
# transform train input data
X_train_fs = f_selector.transform(X_train)
# transform test input data
X_test_fs = f_selector.transform(X_test)
# Plot the scores for the features
plt.bar([i for i in range(len(f_selector.scores_))], f_selector.scores_, color = 'purple')
plt.xlabel("feature index")
plt.ylabel("F-value (transformed from the correlation values)")
plt.title('F-value by Feature')
plt.show()

#Feature Selection for Mutual Information

# feature selection
f_selector = SelectKBest(score_func=mutual_info_regression, k='all')
# learn relationship from training data
f_selector.fit(X_train, y_train)
# transform train input data
X_train_fs = f_selector.transform(X_train)
# transform test input data
X_test_fs = f_selector.transform(X_test)
# Plot the scores for the features
plt.bar([i for i in range(len(f_selector.scores_))], f_selector.scores_, color = 'green')
plt.xlabel("feature index")
plt.ylabel("Estimated MI value")
plt.title("Estimated Mutual Information by Feature")

plt.show()

#The y-axis represents the estimated mutual information between each feature and the target variable

In [ ]:
#Step 4 Reduce Size of X_train to only select n best features -- Using selectKbest to do so

# Create and fit selector
selector = SelectKBest(f_regression, k=10)
selector.fit(X_train, y_train)

# Get columns to keep and create new dataframe with those only
cols_idxs = selector.get_support(indices=True)
X_train_new = X_train.iloc[:,cols_idxs]

In [ ]:
#Step 5:
#Pull out the k best chosen features using their column names and append the scoring type column to that list
cols_list = X_train_new.columns.tolist()
cols_list.append('fantasy_points')

#Create df for correlation heatmap using selected features and scoring column above -- focusing on correlation to score column
correlation_df = vet_qb_df[cols_list]

#This is primarily just for visualization purposes for the blog report

In [ ]:
#Step 6 Use correlation df above to get correlation heatmap for given reduced df

plt.figure(figsize=(16, 6))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(correlation_df.corr(), vmin=-1, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
#Scatter Plot of Predictions Vs Actual

#Example: Veteran Qbs in Standard Scoring

#Veteran QBs
vet_qb_X = vet_qb_df.iloc[:,:-3]
vet_qb_y_stand = vet_qb_df['fantasy_points']

#Split into train and test sets for each of the 24 dataframes
X_train, X_test, y_train, y_test = train_test_split(vet_qb_X, vet_qb_y_stand, test_size=0.2, random_state=28)

test_pred = predictions['vet-fantasy_points-QB']

plt.scatter(y_test, test_pred)
m, b = np.polyfit(y_test, test_pred, 1)
#use black as color for regression line
plt.plot(y_test, m*y_test+b, color='black')

plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values -- Vet QBs Standard Scoring')
plt.show()